# YouTube Spam Collection v. 1

## Grupo 2
Alberto Atilio Sbrana Junior
<br>
Luiz Barreto Pedro de Alcântara
<br>
Priscila Portela Costa

# Problema

Trata-se de um problema de classificação binária sobre comentários de vídeos no Youtube.
<br>
Há no total 5 arquivos, separados por artista:
- Psy
- Katy Perry
- LMFAO
- Eminem
- Shakira

# Bibliotecas

In [463]:
import pandas as pd
import numpy as np

import svmutil
from svmutil import svm_read_problem
from svmutil import svm_problem
from svmutil import svm_parameter
from svmutil import svm_train
from svmutil import svm_predict
from svmutil import svm_save_model
import matplotlib.pyplot as plt #visualização
from ML_library import *

# Dataset final

In [464]:
df_psy = pd.read_csv('Youtube 01-comments Psy.csv')
df_kat = pd.read_csv('Youtube 04-comments KatyPerry.csv')
df_lma = pd.read_csv('Youtube 07-comments LMFAO.csv')
df_emi = pd.read_csv('Youtube 08-comments Eminem.csv')
df_sha = pd.read_csv('Youtube 09-comments Shakira.csv')

In [465]:
df_psy.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1


# Tratamento de dados - Occurrency

In [466]:
#origem do video
df_psy['origin'] = 'psy'
df_kat['origin'] = 'kat'
df_lma['origin'] = 'lma'
df_emi['origin'] = 'emi'
df_sha['origin'] = 'sha'

#junção dos datasets
df_start = df_psy.append([df_kat, df_lma, df_emi, df_sha]).reset_index(drop=True)

#criação de novas colunas: ano, data e has_date (booleana, para o caso do dataset do eminem)
df_start['DATE'] = pd.to_datetime(df_start['DATE'],infer_datetime_format=True)
df_start['comment_year'] = df_start['DATE'].dt.year
df_start['comment_hour'] = df_start['DATE'].dt.hour
df_start['has_date'] = np.where(df_start['comment_year'] == 1969, False, True)
df_start['comment_len'] = df_start['CONTENT'].str.len()

#apagar colunas
df_start.drop(['DATE', 'COMMENT_ID'], axis=1, inplace=True)
df_start.drop_duplicates(inplace=True)

In [467]:
df_start.head()

,AUTHOR,CONTENT,CLASS,origin,comment_year,comment_hour,has_date,comment_len
0,Julius NM,"Huh, anyway check out this you[tube] channel: ...",1,psy,2013.0,6.0,True,56
1,adam riyati,Hey guys check out my new channel and our firs...,1,psy,2013.0,12.0,True,166
2,Evgeny Murashkin,just for test I have to say murdev.com,1,psy,2013.0,17.0,True,38
3,ElNino Melendez,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1,psy,2013.0,8.0,True,48
4,GsMega,watch?v=vtaRGgvGtWQ Check this out .﻿,1,psy,2013.0,16.0,True,39


In [468]:
#dicionario pra substituicao de contracoes, girias, typos, stemming
word_dict = {
    'pls': 'please',
    'plz': 'please',
    'plizz': 'please',
    'pplease': 'please',
    'pleassssssssssssssss': 'please',
    'dis': 'this',
    'yt': 'youtube',
    'you[tube]': 'youtube',
    'instagraml': 'instagram',
    'facebook-page': 'facebook',
    'wiredo': 'wierdo',
    'vid': 'video',
    'videoeo': 'video',
    'sub': 'subscribe',
    'thx': 'thanks',
    'suscribe': 'subscribe',
    'allot': 'lot',
    'wat': 'what',
    'should.d': 'should',
    'ilove': 'love',
    'likesubscribescribe': 'subscribe',
    'subscribescribe': 'subscribe',
    'anyoutubehing' : 'youtube',
    'itttttttt': 'it',
    'im': 'i',
    'channelthanks': 'channel'
}

#lista de stop workds da biblioteca NLTK
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves',
              'you','you are','you have','you will',"you had",'your','yours','yourself','yourselves',
              'he','him','his','himself','she',"she is",'her','hers','herself','it',"it is",'its',
              'itself','they','them','their','theirs','themselves','what','which','who','whom','this','that',
              "that will",'these','those','am','is','are','was','were','be','been','being','have',
              'has','had','having','do','does','did','doing','a','an','the','and','but','if',
              'or','because','as','until','while','of','at','by','for','with','about','against','between',
              'into','through','during','before','after','above','below','to','from','up','down',
              'in','out','on','off','over','under','again','further','then','once','here','there',
              'when','where','why','how','all','any','both','each','few','more','most','other',
              'some','such','no','nor','not','only','own','same','so','than','too','very','s','t','can',
              'will','just','don',"do not",'should',"should have",'now','d','ll','m','o','re','ve','y','ain',
              'aren',"are not",'couldn',"could not",'didn',"did not",'doesn',"does not",'hadn',"had not",
              'hasn',"has not",'haven',"have not",'isn',"is not",'ma','mightn',"might not",'mustn',"must not",
              'needn',"need not",'shan',"shall not",'shouldn',"should not",'wasn',"was not",'weren',"were not",
              'won',"will not",'wouldn',"would not", 'ill']

In [469]:
def trata_dados(df, treatment_type):
    
    #transformação trigonometrica da coluna hour
    df['sin_time'] = np.sin(2*np.pi*df['comment_hour']/24)
    df['cos_time'] = np.cos(2*np.pi*df['comment_hour']/24)
        
    #criar dummy pra cada origem
    df = pd.get_dummies(df, columns=['origin', 'has_date'])
    
    #novas feature
    df['has_www'] = df['CONTENT'].str.contains(pat = 'www') 
    df['has_http'] = df['CONTENT'].str.contains(pat = 'http')
    df['has_link'] = df['CONTENT'].str.contains(pat = 'watch\?v=')   
    df['has_website'] = (df['has_www'] == True) | (df['has_http'] == True) | (df['has_link'] == True)
    
    #tratamento de texto
    df.drop('AUTHOR', axis=1, inplace=True)
    df['CONTENT'] = df['CONTENT'].str.lower()
    
    column_list = []
    
    for index, row in df.iterrows():
        content = row['CONTENT']
        
        #retirar emojis e caracteres especiais
        regular_characters = "abcdefghijklmnopqrstuvwxyz " 
        content = ''.join(c for c in content if c in regular_characters)
        
        #transfoma texto em lista de palavras:
        content = content.split()
                
        #substituir contracoes de palavras
        for k, v in word_dict.items():
            content = [c.replace(k, v) for c in content]
        
        #remover stop words
        for sw in stop_words:
            content = [c for c in content if sw != c]
        
        column_list.append(content)
        
    column_list = [word for word in column_list if len(word) > 0]
 
    df['CONTENT'] = pd.Series(column_list)
    
    df.dropna(axis=0, subset=['CONTENT'], inplace=True)
    df = df[df['CONTENT'] != '[]']
    
    #transformar cada palavra em uma feature
    columns = []
    for sublist in column_list:
        for item in sublist:
            columns.append(item) 
        
    columns = set(columns)            
    dict_list = []
        
    #valores em cada coluna
    for index, row in df.iterrows():
        content_words = row['CONTENT']
        
        bag_of_words_dict = {}       
        for c in columns:
            bag_of_words_dict[c] = 0
        
        for w in content_words:
            if treatment_type == 'occurrency':
                bag_of_words_dict[w] = 1
            if treatment_type == 'frequency':
                bag_of_words_dict[w] += 1
            if treatment_type == 'tf-idf':         
                bag_of_words_dict[w] +=1
                
        if treatment_type == 'tf-idf': 
            for w in bag_of_words_dict:
                bag_of_words_dict[w] = bag_of_words_dict[w] / len(content_words)
        
        dict_list.append(bag_of_words_dict)        
     
    df = df.assign(bag_words=dict_list)
    bag_words = df['bag_words'].apply(pd.Series)
    
    if treatment_type == 'tf-idf':
        for c in bag_words:
            if bag_words[c].sum() > 0:
                bag_words[c] = np.log(len(df)/bag_words[c].sum())*bag_words[c]
            else:
                bag_words[c] = 0
        
    df = pd.concat([df, bag_words], axis = 1)
    
    #remover coluna content
    df.drop(['CONTENT','bag_words'], axis=1, inplace=True)
    #remover possiveis nulos
    df.dropna(inplace=True)
     
    return df, columns

In [470]:
df_treated, columns = trata_dados(df_start, treatment_type='occurrency')

In [471]:
df_treated.head()

,CLASS,comment_year,comment_hour,comment_len,sin_time,cos_time,origin_emi,origin_kat,origin_lma,origin_psy,...,screw,society,sexy,subscribeeeeeeeeee,courtthanks,briefs,low,china,httpenprothomalocomsportnewszibabweateamdueindhakawednesday,man
0,1,2013.0,6.0,56,1.000000e+00,6.123234e-17,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,2013.0,12.0,166,1.224647e-16,-1.000000e+00,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,2013.0,17.0,38,-9.659258e-01,-2.588190e-01,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,2013.0,8.0,48,8.660254e-01,-5.000000e-01,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
4,1,2013.0,16.0,39,-8.660254e-01,-5.000000e-01,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [472]:
X = df_treated.drop('CLASS', axis=1)
Y = df_treated['CLASS']

# Modelo Escolhido para implementação : Support Vector Machines

## Funções auxiliares

In [473]:
def prep_Y(y):
    return np.array(y.values.flatten(), dtype = int)

def min_max_scaler(x, scaler = None):
    if scaler == None:
        x_scaler = [x.min(), x.max()]
        x_scaled = (x - x.min()) / (x.max() - x.min())
        x_scaled.fillna(0, inplace = True)
        return x_scaler, x_scaled
    else:
        x_scaled = (x - scaler[0]) / (scaler[1] - scaler[0]).replace(0, np.inf)
        x_scaled = x_scaled.replace([np.inf, -np.inf], np.nan).fillna(0)
        return x_scaled

## Occurency: C=3.000, gamma=0.100

In [474]:
Y_prep = prep_Y(Y)

scaler, X_scaled = min_max_scaler(X)
X_scaled_v = X_scaled.values

In [475]:
custo = 3
gamma = 0.1
kernel = 2

model = svm_train(Y_prep, X_scaled_v, '-c %f -t %d -g %f' %(custo, kernel, gamma))

# Preparação de um dado de entrada novo:

In [495]:
predict_row = {
    'COMMENT_ID': ['vbcido mxnoinicn'],
    'AUTHOR': ['Priscila Portela'],
    'DATE': ['2013-11-07T06:20:48'],
    #'CONTENT': ['Hey guys check out my new channel'], #SPAM
    #'CONTENT': ['free money www'], #SPAM
    #'CONTENT': ['GOOD MUSIC'], #HAM
    'CONTENT': ['check out my youtube channel subscribe pls'], #SPAM
    #'CONTENT': ['check http'], #SPAM
    #'CONTENT' : ['i love this song'], #HAM
    'origin': 'psy'
}

predict_row = pd.DataFrame.from_dict(predict_row, orient='columns')
predict_row

,COMMENT_ID,AUTHOR,DATE,CONTENT,origin
0,vbcido mxnoinicn,Priscila Portela,2013-11-07T06:20:48,check out my youtube channel subscribe pls,psy


## Tratamento do dado de entrada novo

In [496]:
#criação de novas colunas: ano, data e has_date (booleana, para o caso do dataset do eminem)
predict_row['DATE'] = pd.to_datetime(predict_row['DATE'],infer_datetime_format=True)
predict_row['comment_year'] = predict_row['DATE'].dt.year
predict_row['comment_hour'] = predict_row['DATE'].dt.hour
predict_row['comment_len'] = predict_row['CONTENT'].str.len()

#apagar colunas
predict_row.drop(['COMMENT_ID'], axis=1, inplace=True)
predict_row.drop_duplicates(inplace=True)

In [497]:
predict_row.head()

,AUTHOR,DATE,CONTENT,origin,comment_year,comment_hour,comment_len
0,Priscila Portela,2013-11-07 06:20:48,check out my youtube channel subscribe pls,psy,2013,6,42


In [498]:
def trata_dados_novos(df, treatment_type):
    
    #transformação trigonometrica da coluna hour
    df['sin_time'] = np.sin(2*np.pi*df['comment_hour']/24)
    df['cos_time'] = np.cos(2*np.pi*df['comment_hour']/24)
        
    df['origin_emi'] = 0
    df['origin_kat'] = 0
    df['origin_lma'] = 0
    df['origin_psy'] = 0
    df['origin_sha'] = 0
        
    if df['origin'][0] == 'emi':
         df['origin_emi'] = 1            
    if df['origin'][0] == 'kat':
         df['origin_kat'] = 1
    if df['origin'][0] == 'lma':
         df['origin_lma'] = 1           
    if df['origin'][0] == 'psy':
         df['origin_psy'] = 1
    if df['origin'][0] == 'sha':
         df['origin_sha'] = 1
    
    df['has_date'] = np.where(df['comment_year'] == 1969, False, True)    
    df.drop(['origin', 'DATE'], axis=1, inplace=True)
        
    #novas feature
    df['has_www'] = df['CONTENT'].str.contains(pat = 'www') 
    df['has_http'] = df['CONTENT'].str.contains(pat = 'http')
    df['has_link'] = df['CONTENT'].str.contains(pat = 'watch\?v=')   
    df['has_website'] = (df['has_www'] == True) | (df['has_http'] == True) | (df['has_link'] == True)
    
    #tratamento de texto
    df.drop('AUTHOR', axis=1, inplace=True)
    df['CONTENT'] = df['CONTENT'].str.lower()
    
    column_list = []
    
    for index, row in df.iterrows():
        content = row['CONTENT']
        
        #retirar emojis e caracteres especiais
        regular_characters = "abcdefghijklmnopqrstuvwxyz " 
        content = ''.join(c for c in content if c in regular_characters)
        
        #transfoma texto em lista de palavras:
        content = content.split()
                
        #substituir contracoes de palavras
        for k, v in word_dict.items():
            content = [c.replace(k, v) for c in content]
        
        #remover stop words
        for sw in stop_words:
            content = [c for c in content if sw != c]
        
        column_list.append(content)
        
    column_list = [word for word in column_list if len(word) > 0]
 
    df['CONTENT'] = pd.Series(column_list)
    
    df.dropna(axis=0, subset=['CONTENT'], inplace=True)
    df = df[df['CONTENT'] != '[]']
             
    dict_list = []
        
    #valores em cada coluna
    for index, row in df.iterrows():
        content_words = row['CONTENT']
        content_words = [x for x in content_words if x in columns]
        
        bag_of_words_dict = {}       
        for c in columns:
            bag_of_words_dict[c] = 0
                 
        for w in content_words:
            if treatment_type == 'occurrency':
                bag_of_words_dict[w] = 1
        
        dict_list.append(bag_of_words_dict)        
     
    df = df.assign(bag_words=dict_list)
    bag_words = df['bag_words'].apply(pd.Series)
        
    df = pd.concat([df, bag_words], axis = 1)
    
    #remover coluna content
    df.drop(['CONTENT','bag_words'], axis=1, inplace=True)
    #remover possiveis nulos
    df.dropna(inplace=True)
    df.fillna(0, inplace=True)
     
    return df

In [499]:
row_treated = trata_dados_novos(predict_row, 'occurrency')

In [500]:
row_treated.head()

,comment_year,comment_hour,comment_len,sin_time,cos_time,origin_emi,origin_kat,origin_lma,origin_psy,origin_sha,...,screw,society,sexy,subscribeeeeeeeeee,courtthanks,briefs,low,china,httpenprothomalocomsportnewszibabweateamdueindhakawednesday,man
0,2013,6,42,1.0,6.123234e-17,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


Agora, o dado de entrada `row_treated` possui as mesmas colunas, na mesma ordem, do conjunto de dados que treinou o modelo.

# Predição do dado novo

In [501]:
row_treated_scaled = min_max_scaler(np.array(row_treated.values.flatten(), dtype = float), scaler).values

In [502]:
def test_new_data(x, model, scaler = None):
    if scaler:
        x = min_max_scaler(np.array(row_treated.values.flatten(), dtype = float), scaler).values
    xclass = svm_predict([], [x], model, options = '-q')[0][0]
    print('O comentário é classificado como {}.'.format('SPAM' if xclass else 'HAM'))

In [503]:
test_new_data(row_treated_scaled, model)

O comentário é classificado como SPAM.
